### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

step 1: compute the normalized loss by dividing the loss with the number of steps
step 2: compute the gradients with respect to the normalized loss
step 3: compute the acumulative gradient by ...
step 4: 

In [ ]:
node > pod > container

In [ ]:
Layer 1: gathered, computed, released
Layer 2: gathered, computed, released
Layer 3: gathering...
Layer 4, 5: Empty

In [ ]:
global memory > shared memory > register memory

In [ ]:
number of chunks

In [ ]:
step 1: normalize the loss
step 2: calculate the grads with respect to the normalized loss
step 3: sum the grads
step 4: if current epoch = number of acc steps, update, otherwise, step 1

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        
        world_size = torch.distributed.get_world_size()
        self.num_embeddings_per_partrition = num_embeddings // world_size
        self.embedding_dim = embedding_dim
        
        self.weight = nn.Parameter(torch.randn(
            self.num_embeddings_per_partrition,
            self.embedding_dim
        ))
        
        self.vocab_start_idx, self.vocab_end_idx = self.get_vocab_range(
            self.num_embeddings_per_partrition
        )
        
    def get_vocab_range(self, num_embeddings_per_partrition):
        rank = torch.distributed.get_rank()
        start_idx = rank*num_embeddings_per_partrition
        end_idx = start_idx + num_embeddings_per_partrition
        return start_idx, end_idx
    
    def forward(self, tokens):
        masks = (tokens < self.vocab_start_idx) | (tokens > self.vocab_end_idx)
        masked_tokens = tokens.clone() - self.vocab_start_idx
        masked_tokens[masks] = 0.
        
        embeddings = F.embedding(masked_tokens, weight=self.weight)
        mask_idxs = torch.where(masks==True)[1]
        embeddings[mask_idxs] = 0.
        
        torch.distributed.all_reduce(embeddings)
        
        return embeddings

In [ ]:
W_normalized = W / W.norm(dim=-1)

In [ ]:
similarities = W_normalized @ W_normalized.T

In [3]:
from transformer_lens.utils import get_act_name

In [ ]:
n_layers = 3

In [ ]:
head_names = [get_act_name("result", layer_idx) for layer_idx in range(n_layers)]
mlp_names = [get_act_name("mlp_out", layer_idx) for layer_idx in range(n_layers)]

In [ ]:
hook_names = ["embed", "pos_embed"] + head_names + mlp_names

In [ ]:
_, cache = model.run_with_cache(
    tokens,
    names_filter=lambda x: x in hook_names
)

In [ ]:
input_components = torch.cat([cache["embed"] + cache["pos_embed"]]).unsqueeze(0)

In [ ]:
for head_name, mlp_name in zip(head_names, mlp_names):
    input_components = torch.cat([
        input_components,
        cache[head_name],
        cache[mlp_name]
    ])

In [ ]:
W_U = model.W_U
logit_diff_direction = W_U[:, 0] - W_U[:, 1]

In [ ]:
pre_ln_dir = coeffs.T @ logit_diff_direction

In [4]:
from einops import einsum

In [ ]:
contributions = einsum(
    input_components,
    pre_ln_dir
)

In [ ]:
step 1: rewrite probs as logit difference
step 2: calculate the logit difference direction
step 3: decompose the logits
step 4: project each component to 

In [5]:
from circuitsvis.attention import attention_patterns

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
attn_pattern = cache["pattern", 0, "attn"]

In [ ]:
str_tokens = model.to_str_tokens(tokens)

In [ ]:
attention_patterns(
    tokens=str_tokens,
    attention=attn_pattern
)

In [ ]:
logits = model(past_moves)

In [ ]:
log_probs = F.log_softmax(logits[:, -1, :], dim=-1)

In [ ]:
board_state = torch.zeros(board_size, board_size)
board_state[]

In [ ]:
W_in = model.W_in
input_direction = W_in[layer_idx, :, neuron_idx]

In [ ]:
W_E = model.W_E
W_Q = model.W_Q[1, 4]
W_K = model.W_K[1, 4]
W_O = model.W_O[0, 7]
W_V = model.W_V[0, 7]

In [ ]:
Q = W_E @ W_Q
K = W_E @ W_V @ W_O @ W_K 

In [ ]:
import os

In [ ]:
step 1: determine number of workers
step 2: flatten the weight
step 3: chunking
step 4: append empty chunk if necessary
step 5: extract shjard
step 5: 

In [6]:
ranks = [0, 1, 3, 6]

In [7]:
group = None

In [ ]:
rank = torch.distributed.get_rank()

In [ ]:
if rank in ranks:
    group = torch.distributed.new_group(ranks=ranks)

In [ ]:
if group is not None:
    torch.distributed.broadcast(x, src=0, group=group)

In [ ]:
max(clock_idx+1-n_microbatches, 0)

In [ ]:
step 1: gather
step 2: 

In [ ]:
elastic driver, host discovery, 3 notification, torchstate

In [ ]:
step 1: partition
step 2: forward pass, gather, do the work, release
step 3:  backward
step 4: reduce-scatter
step 5: update

In [ ]:
ready, running, suceed, failed, cooldown, blacklisted

In [8]:
import threading

In [ ]:
t1 = threading.Thread(target=)

In [9]:
{x for x in "hello"}

{'e', 'h', 'l', 'o'}

In [10]:
import threading

In [ ]:
lock = threading.Lock()

In [ ]:
t1 = threading.Thread(target=print_numbers)

In [ ]:
with lock:
    t1.start()

In [11]:
from torch.utils.data import DataLoader, random_split

In [ ]:
train_set, valid_set = random_split(dataset)

In [ ]:
train_loader = DataLoader(train_set, batch_size=2)
valid_loader = DataLoader(valid_set, batch_size=2)

In [ ]:
W_out[i, :]

In [ ]:
step 1: two prompts
step 2: record the interdimate activations
step 3: iteratively patch
step 4: compute the logit difference

In [ ]:
W_E@W_Q^{1, 4}@W_K^{1, 4}@W_OV^{0, 7}@W_E

In [ ]:
ReLU(x@W_in)@W_out

sum of d_mlp neurons(ReLU(x@W_in[:, i]))@W_out[i, :]

In [ ]:
W_in[:, i]

In [ ]:
nn.Flatten(start_dim=2, end_dim=3)

In [ ]:
host discovery, 3 notif, torchstate, elastic driver

In [ ]:
elastic driver, torchstate, 3 notif, script

In [ ]:
text_similarities = text_embeddings @ text_embedding.T
image_similarities = image_embedding @ image_embedding.T

In [ ]:
targets = F.softmax(
    (text_similarities + image_similarities) / 2,
    dim=-1
)

In [ ]:
step 1: number of worker nodes
step 2: flatten 10 * 4 = 40
step 3: chunk
step 4: add empty
step 5: shard
step 6: padd

In [ ]:
for _ in range(10):
    action = env.action.sample()
    env.step(action)

In [12]:
from torch.nn.utils import clip_grad_norm_

In [ ]:
clip_grad_norm_(param_1, max_norm=GRAD)

In [ ]:
loss = F.kl_div(
    input=F.log_softmax(student_logits),
    target=F.softmax(teacher_logits)
)